In [1]:
import pandas  as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
df = pd.read_csv("C:\\Datasets\\parkinson\\parkinsons.csv")

In [3]:
df.isnull().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [4]:
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [5]:
df['name'].nunique()

195

In [6]:
df.shape

(195, 24)

# EXTREME LEARNING IMPLEMENTATION

In [7]:
# Generating Weights for our inputs as a Matrix

def create_one_hot_encoding(classes, shape):
    one_hot_encoding = np.zeros(shape)
    for i in range(0, len(one_hot_encoding)):
        one_hot_encoding[i][int(classes[i])] = 1
    return one_hot_encoding


In [8]:
# Calculating Hidden_Layer Output Matrix and the Moore-Penrose inverse
def train(weights,x,y) :
    h = x.dot(weights)
    h = np.maximum(h,0,h)
    return np.linalg.pinv(h).dot(y)


In [9]:
# Activating the Output Matrix using softmax
def softmax(layer) :
    soft_max_output_layer = np.zeros(len(layer))
    for i in range(0,len(layer)):
        numitor = 0
        for j in range(0,len(layer)):
            numitor+=np.exp(layer[j]-np.max(layer))
            soft_max_output_layer[i] = np.exp(layer[i]-np.max(layer))/numitor
    return soft_max_output_layer

def matrix_softmax(matrix_) :
    soft_max_matrix = []
    for i in range(0,len(matrix_)):
        soft_max_matrix.append(softmax(matrix_[i]))
    return soft_max_matrix

In [10]:
# Checking Network Power 
def check_network_power(o, o_real):
    count = 0
    for i in range(0, len(o)):
        count += 1 if np.argmax(o[i]) == np.argmax(o_real[i]) else 0
    return count

In [11]:
# Testing our predictions
def test (weights,beta,x,y):
    h = x.dot(weights)
    h = np.maximum(h,0,h)
    o = matrix_softmax(h.dot(beta))
    return check_network_power(o, y)/len(y)

# ACCURACY TEST::

In [12]:
df.drop(['name'],axis=1,inplace=True)

In [13]:
df.columns

Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [37]:
class_column = 16
test_size = 0.1
db = df.iloc[:, :].values.astype(float)
#np.random.shuffle(db)
y = db[:, class_column]
y -= np.min(y)
output_layer_perceptron_count = len(np.unique(y))
y = create_one_hot_encoding(y, (len(y), len(np.unique(y))))
x = np.delete(db, [class_column], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
hidden_layer_perceptron_count = len(y_test)
x = preprocessing.normalize(x)
weights = np.random.random((len(x[0]), hidden_layer_perceptron_count))
beta = train(weights, x_train, y_train)
print("Accuracy = %s." % test(weights, beta, x_test, y_test))

Accuracy = 0.95.


 FOR OUR DATASET :THE ACCURACY WAS VERY HIGH BECAUSE OF THE STATISTICAL POWER OF OUR ALGORITHM
 Extreme Learning Machines are a good alternative to other Neural Networks

# THE END :: THANK YOU